In [20]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import os

his_data1 = pd.read_csv("../ask_px_diff/GCG0%3AMBO.csv")
his_data1.drop("datetime",1,inplace=True)
his_data1 = his_data1[0:0]

s=1
for file in os.listdir("../ask_px_diff"):
    data = pd.read_csv("../ask_px_diff/"+file)
    print(file.replace("%3",":")[:-4])
    data.drop("datetime",1,inplace=True)
    data.insert(0,column="security",value=s)
    his_data1 = pd.concat([data,his_data1])
    s+=1
his_data1.rename(columns={ his_data1.columns[-1]: 'mkt' }, inplace = True)
his_data1['time'] = his_data1['time'].apply(lambda x: int(x[-8:-7])*60+int(x[-5:-4]))


his_data2 = pd.read_csv("../bid_px_diff/GCG0%3AMBO.csv")
his_data2.drop("datetime",1,inplace=True)
his_data2 = his_data2[0:0]

s=1
for file in os.listdir("../bid_px_diff"):
    data = pd.read_csv("../bid_px_diff/"+file)
    #s = file.replace("%3",":")[:-4]
    data.drop("datetime",1,inplace=True)
    data.insert(0,column="security",value=s)
    his_data2 = pd.concat([data,his_data2])
    s+=1
his_data2.rename(columns={ his_data2.columns[-1]: 'mkt' }, inplace = True)
his_data2['time'] = his_data2['time'].apply(lambda x: int(x[-8:-7])*60+int(x[-5:-4]))

GCG0:AMBO
GEH0:AMBO
GEH1:AMBO
GEH2:AMBO
GEH3:AMBO
GEH4:AMBO
GEM0:AMBO
GEM1:AMBO
GEM2:AMBO
GEM3:AMBO
GEM4:AMBO
GEU0:AMBO
GEU1:AMBO
GEU2:AMBO
GEU3:AMBO
GEZ0:AMBO
GEZ1:AMBO
GEZ2:AMBO
GEZ3:AMBO
TNH0:AMBO
UBH0:AMBO
ZBH0:AMBO
ZFH0:AMBO
ZNH0:AMBO
ZTH0:AMBO


In [4]:
his_data1.shape

(32735, 13)

In [5]:
his_data2.shape

(32735, 13)

In [6]:
his_data2

,security,time,L1bid_price,L1bid_size,L2bid_price,L2bid_size,L3bid_price,L3bid_size,L4bid_price,L4bid_size,L5bid_price,L5bid_size,mkt
0,25.0,123,107.718750,76.000000,107.710938,30.000000,107.707031,451.000000,107.703125,633.000000,107.699219,505.000000,0
1,25.0,124,107.718750,76.000000,107.710938,30.000000,107.707031,342.000000,107.703125,623.000000,107.699219,505.000000,0
2,25.0,124,107.718750,76.000000,107.710938,30.000000,107.707031,342.000000,107.703125,623.000000,107.699219,505.000000,0
3,25.0,125,107.722656,6.000000,107.718750,86.000000,107.710938,22.000000,107.707031,184.000000,107.703125,225.000000,1
4,25.0,120,107.709080,124.754098,107.705174,269.868852,107.701268,354.278689,107.697362,395.885246,107.693455,401.672131,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377,1.0,125,15180.000000,8.384615,15179.000000,19.000000,15178.000000,19.000000,15177.000000,21.076923,15176.000000,13.000000,0
1378,1.0,125,15180.444444,6.611111,15179.444444,14.111111,15178.444444,19.333333,15177.444444,19.000000,15176.444444,14.833333,0
1379,1.0,125,15180.000000,4.363636,15179.000000,17.772727,15178.000000,15.363636,15177.000000,15.045455,15176.000000,14.545455,0
1380,1.0,125,15179.576923,11.769231,15178.576923,13.153846,15177.576923,12.653846,15176.576923,12.769231,15175.576923,13.769231,0


In [7]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report, accuracy_score

In [8]:
import warnings
warnings.simplefilter('ignore')

In [9]:
from sklearn.model_selection import train_test_split
X = his_data2.drop(['mkt'], axis = 1).values
y = his_data2['mkt'].values

seed = 1

# Splitting data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = seed)

In [11]:
scoring = 'accuracy'
# Model building to train
names = ['K Nearest Neighbors', 'Decision Tree', 'Random Forest', 'Neural Net', 'AddaBoost', 'Naive Bayes']

Classifiers = [
    KNeighborsClassifier(n_neighbors = 3),
    DecisionTreeClassifier(max_depth = 5),
    RandomForestClassifier(max_depth = 5, n_estimators = 10, max_features = 1 ),
    MLPClassifier(alpha = 1),
    AdaBoostClassifier(),
    GaussianNB(), 
    ]

models = zip(names, Classifiers)

from sklearn.model_selection import KFold, cross_val_score

names = []
result = []
for name, model in models:
    kfold = KFold(n_splits = 10, random_state = 1, shuffle = True)
    cv_results = cross_val_score(model, X_train, y_train, cv = kfold, scoring = 'accuracy')
    result.append(cv_results)
    names.append(name)
    msg = "{0}: {1} ({2})".format(name, cv_results.mean(), cv_results.std())
    print(msg)

K Nearest Neighbors: 0.820862429264215 (0.00854337642572351)
Decision Tree: 0.8500671202160055 (0.005474680003213438)
Random Forest: 0.8457497163934535 (0.006086983507477675)
Neural Net: 0.7078284362830626 (0.1332103970090113)
AddaBoost: 0.8342227356256261 (0.006745013534371254)
Naive Bayes: 0.6845754857798683 (0.006600482811215266)


In [12]:
#Test the algorithm on the test data set
models = zip(names, Classifiers)
for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(name)
    print(accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    

K Nearest Neighbors
0.820136852394917
              precision    recall  f1-score   support

          -1       0.21      0.14      0.17       640
           0       0.87      0.94      0.91      6903
           1       0.38      0.18      0.25       641

    accuracy                           0.82      8184
   macro avg       0.49      0.42      0.44      8184
weighted avg       0.78      0.82      0.80      8184

Decision Tree
0.8492179863147605
              precision    recall  f1-score   support

          -1       0.33      0.00      0.01       640
           0       0.86      0.99      0.92      6903
           1       0.55      0.17      0.26       641

    accuracy                           0.85      8184
   macro avg       0.58      0.39      0.40      8184
weighted avg       0.79      0.85      0.80      8184

Random Forest
0.843475073313783
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       640
           0       0.84   

In [13]:
model = DecisionTreeClassifier(max_depth = 5)
model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=5)

In [15]:
import pickle
filename = 'DecisionTree_bid.sav'
#pickle.dump(model, open(filename, 'wb'))

In [16]:
model = pickle.load(open(filename, 'rb'))
result = model.score(X_test, y_test)
y_pred = model.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [17]:
y_pred = model.predict(X_test[0].reshape(1,-1))
y_pred

array([0], dtype=int64)

In [19]:
X_test[0]

array([7.00000000e+00, 6.00000000e+01, 9.83050000e+03, 3.42359444e+04,
       9.83000000e+03, 2.31246296e+03, 9.82950000e+03, 9.82055556e+02,
       9.82900000e+03, 9.41000000e+02, 9.82850000e+03, 9.58000000e+02])